In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
from shapely.geometry import LineString
from shapely.ops import split, linemerge
import krippendorff

In [2]:
from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

In [3]:
#split line into segments of 30 ft
def split_line_at_intervals(line, interval=30):
    distance = 0
    segments = []

    while distance < line.length:
        sub_line = line.interpolate(distance)
        next_distance = distance + interval
        next_sub_line = line.interpolate(next_distance)
        segment = LineString([sub_line, next_sub_line])
        segments.append(segment)
        distance = next_distance

    return segments

def split_geodataframe(geodf, local_crs, interval=30):
    
    geodf = geodf.to_crs(local_crs)
    
    new_rows = []

    for index, row in geodf.iterrows():
        line = row.geometry
        splitted_lines = split_line_at_intervals(line, interval)

        for segment_id, segment in enumerate(splitted_lines):
            new_row = row.copy()
            new_row.geometry = segment
            new_row['segment_id'] = segment_id
            new_rows.append(new_row)

    df = gpd.GeoDataFrame(new_rows, columns=geodf.columns.append(pd.Index(['segment_id'])))
    #add a column that combines street_edge_id and segment_id
    df['street_edge_segment_id'] =  df['street_edge_id'].astype(str) + '_' +  df['segment_id'].astype(str)
    df.crs = local_crs
    return df

In [4]:
#read in chicago validation labels
chicago_labels = gpd.read_file('data/chicago-validation-labels-round2-230608.geojson')
chicago_labels = chicago_labels.drop(columns=['time_created'])
chicago_labels.head()

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
0,Devon Snyder,2548,1259,Milton,CurbRamp,41.893814,-88.103554,JqRFV1QowOOjFb-TIMnzMw,None,1.0,f,None,debris,POINT (-88.10355 41.89381)
1,Devon Snyder,2549,1259,Milton,SurfaceProblem,41.893818,-88.103470,7aric67UimoidY9pcCuWiQ,None,1.0,f,grass,None,POINT (-88.10347 41.89382)
2,Devon Snyder,2550,1259,Milton,SurfaceProblem,41.893982,-88.103340,xzribizdMX2eFxY6W8c7Tg,None,2.0,f,"cracks,grass",None,POINT (-88.10334 41.89398)
3,Devon Snyder,2551,1259,Milton,SurfaceProblem,41.894131,-88.103302,JS7TZGSTE9PJI_DpazVx2g,None,1.0,f,grass,None,POINT (-88.10330 41.89413)
4,Devon Snyder,2552,1259,Milton,SurfaceProblem,41.894184,-88.103271,JS7TZGSTE9PJI_DpazVx2g,None,2.0,f,"grass,bumpy",None,POINT (-88.10327 41.89418)


In [5]:
#read in chicago streets
chicago_streets = gpd.read_file("data/chicago-streets-230608.geojson")

In [6]:
#obtain cluster to street segment mapping
def cluster_to_street_segment_mapping(df,streets_seg):
     # Group by cluster_id and combine all geometries in each group into a MultiPoint
     df_grouped = df.groupby('cluster_id')['geometry'].apply(lambda x: x.unary_union)
     # Calculate centroid of each group
     centroids = df_grouped.centroid
     # Convert centroids to GeoDataFrame
     cluster_centroids = gpd.GeoDataFrame(geometry=centroids).reset_index()
     cluster_centroids.crs = {'init' :'epsg:4326'}
     cluster_centroids_ft = cluster_centroids.to_crs(epsg=streets_seg.crs.to_epsg())
     #map gdf_centroids to chicago_split, delete index_right column
     cluster_centeroids_street = cluster_centroids_ft.sjoin_nearest(streets_seg, distance_col='distance')
     cluster_centeroids_street = cluster_centeroids_street.drop(columns=['index_right'])
     #select only cluster_id and street_edge_segment_id
     cluster_centeroids_street = cluster_centeroids_street[['cluster_id','street_edge_segment_id']]
     #if there are multiple street_edge_segment_ids for a cluster_id, select the first one
     cluster_centeroids_street = cluster_centeroids_street.groupby('cluster_id').first().reset_index()
     return cluster_centeroids_street

In [7]:
#construct agreement table
def irr_table (df,street_seg):
    mikey = df[df['username'] == 'mikey']
    devon = df[df['username'] == 'Devon Snyder']
    irr_mikey = mikey.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    irr_devon = devon.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    #replace usernames with 1
    irr_mikey  = irr_mikey .replace('mikey',1)
    irr_devon  = irr_devon .replace('Devon Snyder',1)
    #group by street_edge_segment_id and count the number of label_ids
    irr_mikey = irr_mikey.groupby('street_edge_segment_id').sum()
    irr_devon = irr_devon.groupby('street_edge_segment_id').sum()
    irr_mikey = irr_mikey[['username']].reset_index()
    irr_devon = irr_devon[['username']].reset_index()
    irr_mikey = irr_mikey.rename(columns={'username':'mikey'})
    irr_devon = irr_devon.rename(columns={'username':'devon'})
    irr_mikey_devon = irr_mikey.merge(irr_devon, on='street_edge_segment_id', how='left').fillna(0)
    irr_mikey_devon = irr_mikey_devon.drop(columns=['street_edge_segment_id'])
    
    return irr_mikey_devon

In [8]:
def krippendorff_alpha(labels,label_type_id, streets, local_crs):
    #clustering by label type
    label_type = cluster_label_type_at_index(labels,label_type_id)[2]
    #split streets into segments
    street_seg = split_geodataframe(streets, local_crs, interval=30)
    #map cluster_id to street_edge_segment_id
    mapping = cluster_to_street_segment_mapping(label_type,street_seg)
    #update the street_edge_segment_id with the one closest to the cluster centroid
    label_type = label_type.merge(mapping, on='cluster_id', how='left')
    #select only the columns we need
    df = label_type[['username','cluster_id','street_edge_segment_id','label_id']]
    #construct agreement table
    irr = irr_table(df,street_seg)
    data_matrix = irr.T.to_numpy()
    alpha = krippendorff.alpha(data_matrix,level_of_measurement='ratio')

    return alpha.round(3)
    

In [9]:
alpha_chicago = {}
for i in [0,1,2,4,5,7,8,9]:
    alpha = krippendorff_alpha(chicago_labels,i, chicago_streets, 3435)
    alpha_chicago[label_types[i]] = alpha

In [10]:
#0608
alpha_chicago

{'CurbRamp': 0.934,
 'NoSidewalk': 0.734,
 'Problem': 0.598,
 'SurfaceProblem': 0.584,
 'Obstacle': 0.595,
 'NoCurbRamp': 0.666,
 'Crosswalk': 0.922,
 'Signal': 0.99}

surface problem and obstacle without severity level =1

In [11]:
#take out label type = surfaceproblem and obstacle
chicago_problems= chicago_labels[(chicago_labels['label_type'] == 'SurfaceProblem') | (chicago_labels['label_type'] == 'Obstacle') ]
#take out severity ==1
chicago_problems = chicago_problems[chicago_problems['severity'] != 1]
chicago_problems

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
2,Devon Snyder,2550,1259,Milton,SurfaceProblem,41.893982,-88.103340,xzribizdMX2eFxY6W8c7Tg,None,2.0,f,"cracks,grass",None,POINT (-88.10334 41.89398)
4,Devon Snyder,2552,1259,Milton,SurfaceProblem,41.894184,-88.103271,JS7TZGSTE9PJI_DpazVx2g,None,2.0,f,"grass,bumpy",None,POINT (-88.10327 41.89418)
19,Devon Snyder,2568,1025,Milton,SurfaceProblem,41.889496,-88.106979,o6Z7NdvLA8c1__0BGK6iXg,None,2.0,f,cracks,None,POINT (-88.10698 41.88950)
29,Devon Snyder,2579,870,Milton,Obstacle,41.889069,-88.106689,G31WYuAr_xrgi2elpukCwg,None,5.0,t,construction,None,POINT (-88.10669 41.88907)
81,Devon Snyder,2658,996,Naperville,SurfaceProblem,41.801285,-88.147972,8JHA5MWiRDo58MSfeimc4w,None,2.0,f,"cracks,uneven/slanted",None,POINT (-88.14797 41.80128)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1673,mikey,5822,867,Downers Grove North,SurfaceProblem,41.773369,-87.982727,96UQoA0lQK2DRdG0vGuzQg,None,3.0,f,uneven/slanted,None,POINT (-87.98273 41.77337)
1674,mikey,5823,1006,Downers Grove North,Obstacle,41.773365,-87.982742,96UQoA0lQK2DRdG0vGuzQg,None,3.0,f,None,None,POINT (-87.98274 41.77337)
1679,mikey,5828,1006,Downers Grove North,SurfaceProblem,41.773376,-87.983231,zYVjcPSOO0cPWqvdc5qhmQ,None,2.0,f,uneven/slanted,None,POINT (-87.98323 41.77338)
1778,Devon Snyder,7947,1256,Milton,Obstacle,41.889648,-88.106621,xOkeRkYjczOgUzVkxthwjw,None,5.0,t,construction,None,POINT (-88.10662 41.88965)


In [12]:
label_types

['CurbRamp',
 'NoSidewalk',
 'Problem',
 'Occlusion',
 'SurfaceProblem',
 'Obstacle',
 'Other',
 'NoCurbRamp',
 'Crosswalk',
 'Signal']

In [13]:
alpha_chicago_problems = {}
for i in [2,4,5]:
    alpha = krippendorff_alpha(chicago_problems,i, chicago_streets, 3435)
    alpha_chicago_problems[label_types[i]] = alpha

In [14]:
alpha_chicago_problems

{'Problem': 0.415, 'SurfaceProblem': 0.359, 'Obstacle': 0.456}

In [15]:
# #plot severity counts for label type = surfaceproblem and obstacle
# sns.countplot(x='severity', hue='label_type', data=chicago_problems)
# #add title
# plt.title('Severity Counts for Surface Problem and Obstacle Labels Chicago')


In [16]:
for i in [0,1,2,4,5,7,8,9]:
    labels= map_clusters(chicago_labels,i)
    map = chicago_streets.explore(m = labels, tiles="CartoDb DarkMatter", color="#BF616A", style_kwds={"stroke":True,"opacity":0.4,"weight": 2},tooltip=["street_edge_id"])
    map.save("html/chicago_" + label_types[i]+ "_230608.html")



In [17]:
for i in [2,4,5]:
    labels= map_clusters(chicago_problems,i)
    map = chicago_streets.explore(m = labels, tiles="CartoDb DarkMatter", color="#BF616A", style_kwds={"stroke":True,"opacity":0.4,"weight": 2},tooltip=["street_edge_id"])
    map.save("html/chicago_wo_sev1_"+label_types[i]+"_230608.html")